# Installation and Setup

In [ ]:
# download the corpus for nltk and spacy's english language model
import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
!pip install pyLDAvis==3.2.2

     |████████████████████████████████| 1.7MB 3.8MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.2-py2.py3-none-any.whl size=135618 sha256=0ed7dae71701208e7057533162dba6296f0cbad165e226c875958610c08c297f
  Stored in directory: /root/.cache/pip/wheels/74/df/b6/97234c8446a43be05c9a8687ee0db1f1b5ade5f27729187eae
Successfully built pyLDAvis


# importing packages

In [ ]:
import re
import numpy as np
import pandas as pd

# GENSIM
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import spacy

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


# importing data

In [ ]:
# importing newsgroup data
from sklearn import datasets
emaildata = datasets.fetch_20newsgroups()
data = emaildata.data

In [ ]:
len(data)

11314

In [ ]:
print(data[11000])

From: yamauchi@ces.cwru.edu (Brian Yamauchi)
Subject: DC-X: Choice of a New Generation (was Re: SSRT Roll-Out Speech)
Organization: Case Western Reserve University
Lines: 27
Distribution: world
NNTP-Posting-Host: yuggoth.ces.cwru.edu
In-reply-to: jkatz@access.digex.com's message of 21 Apr 1993 22:09:32 -0400

In article <1r4uos$jid@access.digex.net> jkatz@access.digex.com (Jordan Katz) writes:

>		   Speech Delivered by Col. Simon P. Worden,
>			The Deputy for Technology, SDIO
>
>	Most of you, as am I, are "children of the 1960's."  We grew
>up in an age of miracles -- Inter-Continental Ballistic Missiles,
>nuclear energy, computers, flights to the moon.  But these were
>miracles of our parent's doing. 

>                          Speech by Pete Worden
>          Delivered Before the U.S. Space Foundation Conference

>     I'm embarrassed when my generation is compared with the last
>generation -- the giants of the last great space era, the 1950's
>and 1960's.  They went to the moon - 

# Data Cleaning

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

## Tokenize words and Clean-up text

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


# Creating Bigram and Trigram Models

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


# Remove Stopwords, Make Bigrams and Lemmatize

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


# Topic Modeling

In [ ]:
# Create Dictionary
dictionary = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]]


In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# View the topics in LDA model

In [ ]:
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.051*"report" + 0.027*"black" + 0.021*"fire" + 0.020*"white" + 0.016*"trial" + 0.016*"cover" + 0.015*"medium" + 0.013*"vote" + 0.012*"minor" + 0.012*"title"'), (1, '0.021*"god" + 0.020*"accept" + 0.016*"member" + 0.015*"man" + 0.014*"israeli" + 0.014*"season" + 0.012*"publish" + 0.012*"lebanese" + 0.012*"jewish" + 0.011*"brain"'), (2, '0.017*"package" + 0.016*"press" + 0.015*"item" + 0.015*"break" + 0.011*"level" + 0.010*"edge" + 0.009*"hole" + 0.007*"eye" + 0.007*"contribute" + 0.007*"equipment"'), (3, '0.025*"pc" + 0.022*"contain" + 0.020*"input" + 0.019*"reality" + 0.017*"picture" + 0.016*"object" + 0.015*"box" + 0.015*"quality" + 0.015*"level" + 0.013*"greek"'), (4, '0.089*"ax" + 0.076*"max" + 0.032*"space" + 0.021*"launch" + 0.018*"di_di" + 0.017*"orbit" + 0.016*"sphere" + 0.015*"satellite" + 0.014*"plane" + 0.014*"mission"'), (5, '0.019*"people" + 0.017*"kill" + 0.015*"child" + 0.015*"government" + 0.012*"attack" + 0.012*"year" + 0.012*"die" + 0.011*"country" + 0.010*"say"

In [ ]:
lda_model.print_topics()

[(0,
  '0.051*"report" + 0.027*"black" + 0.021*"fire" + 0.020*"white" + 0.016*"trial" + 0.016*"cover" + 0.015*"medium" + 0.013*"vote" + 0.012*"minor" + 0.012*"title"'),
 (1,
  '0.021*"god" + 0.020*"accept" + 0.016*"member" + 0.015*"man" + 0.014*"israeli" + 0.014*"season" + 0.012*"publish" + 0.012*"lebanese" + 0.012*"jewish" + 0.011*"brain"'),
 (2,
  '0.017*"package" + 0.016*"press" + 0.015*"item" + 0.015*"break" + 0.011*"level" + 0.010*"edge" + 0.009*"hole" + 0.007*"eye" + 0.007*"contribute" + 0.007*"equipment"'),
 (3,
  '0.025*"pc" + 0.022*"contain" + 0.020*"input" + 0.019*"reality" + 0.017*"picture" + 0.016*"object" + 0.015*"box" + 0.015*"quality" + 0.015*"level" + 0.013*"greek"'),
 (4,
  '0.089*"ax" + 0.076*"max" + 0.032*"space" + 0.021*"launch" + 0.018*"di_di" + 0.017*"orbit" + 0.016*"sphere" + 0.015*"satellite" + 0.014*"plane" + 0.014*"mission"'),
 (5,
  '0.019*"people" + 0.017*"kill" + 0.015*"child" + 0.015*"government" + 0.012*"attack" + 0.012*"year" + 0.012*"die" + 0.011*"count

# Compute Model Coherence Score

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

KeyboardInterrupt: ignored

# Visualize the topics-keywords

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.338785  0.015727       1        1  24.175810
7     -0.310562 -0.102069       2        1  15.735833
17    -0.229598  0.076681       3        1  12.438688
5     -0.141260 -0.192193       4        1   5.439877
9     -0.050106  0.253609       5        1   4.779471
6     -0.022231  0.286013       6        1   3.871266
19     0.021338 -0.011829       7        1   3.288374
8      0.040231 -0.001151       8        1   3.248170
18    -0.051460 -0.014492       9        1   3.240233
1      0.023332 -0.129415      10        1   3.160369
12     0.063571  0.065021      11        1   2.905743
2      0.114133 -0.004402      12        1   2.659831
16     0.055584 -0.140841      13        1   2.353148
10     0.111068 -0.040919      14        1   2.167395
3      0.126622  0.021171      15        1   2.005308
14     0.095816 -0.076092      16        1   1.896412
4      0.131634  0.017184      17        1   1.752177
11     0.108802  0.029884      18        1   1.743140
0      0.121497 -0.036917      19        1   1.633514
15     0.130372 -0.014969      20        1   1.505244, topic_info=           Term          Freq         Total Category  logprob  loglift
19         line  11974.000000  11974.000000  Default  30.0000  30.0000
197       write  10060.000000  10060.000000  Default  29.0000  29.0000
174       would  12046.000000  12046.000000  Default  28.0000  28.0000
139      people   5647.000000   5647.000000  Default  27.0000  27.0000
177     article   6479.000000   6479.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
3369       seat    121.922305    145.119626  Topic20  -4.9374   4.0220
756    purchase    167.433180    311.138771  Topic20  -4.6202   3.5766
521        lead    254.313577    862.156834  Topic20  -4.2022   2.9753
205   condition    153.651212    421.655576  Topic20  -4.7061   3.1867
989        stop    136.600738    898.805143  Topic20  -4.8237   2.3122

[946 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
4621      6  0.991801  accelerator
1211      6  0.015121       accept
1211     10  0.984258       accept
81        3  0.007226       access
81        6  0.384790       access
...     ...       ...          ...
40        8  0.006502         year
40        9  0.371738         year
40       16  0.003110         year
720       9  0.990633    yesterday
570      14  0.998661        young

[1425 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 8, 18, 6, 10, 7, 20, 9, 19, 2, 13, 3, 17, 11, 4, 15, 5, 12, 1, 16])

# LDA Mallet Model

You can achive better performance with LDA Mallet model

In [ ]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = r"D:\AI\jpmc_mumbai\mallet-2.0.8\mallet-2.0.8\bin\mallet" # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=dictionary)


In [ ]:
# Show Topics
print(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

# finding the optimal number of topics for LDA?

Choosing a ‘k’ that marks the end of a rapid growth of topic coherence usually offers meaningful and interpretable topics. Picking an even higher value can sometimes provide more granular sub-topics.

If you see the same keywords being repeated in multiple topics, it’s probably a sign that the ‘k’ is too large.

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()